<a href="https://colab.research.google.com/github/Untick/InspectrumClinic_RS_gr1/blob/main/Marfida%20Alexander/A_Marfida_stajirovka_profpatolog_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Стажировка. Профпатолог (INSPECTRUM CLINIC)
- Подготовка датасетов обучения

# ЗАГРУЗКА ДАТАФРЕЙМА ИСХОДНЫХ ДАННЫХ

In [ ]:
# Подключение библиотек

# Работа с массивами данных
import numpy as np

# Работа с табличными данными
import pandas as pd

# Функции-утилиты для работы с категориальными данными
from tensorflow.keras import utils

# Класс для конструирования последовательной модели нейронной сети
from tensorflow.keras.models import Sequential, Model

# Основные слои
from tensorflow.keras.layers import Dense, Dropout, SpatialDropout1D, BatchNormalization, Embedding, Flatten, Activation, Input, concatenate
from tensorflow.keras.layers import SimpleRNN, GRU, LSTM, Bidirectional, Conv1D, MaxPooling1D, GlobalMaxPooling1D

# Оптимизаторы
from tensorflow.keras.optimizers import Adam, Adadelta, SGD, Adagrad, RMSprop

# Токенизатор для преобразования текстов в последовательности
from tensorflow.keras.preprocessing.text import Tokenizer

# Масштабирование данных
from sklearn.preprocessing import StandardScaler

# Загрузка датасетов из облака google
import gdown

# Регулярные выражения
import re

# Отрисовка графиков
import matplotlib.pyplot as plt
plt.style.use('dark_background')

# Создание статистических графиков
import seaborn as sns

# Табличные данные
from fastai.tabular import *

# Метрики для расчета ошибок
from sklearn.metrics import mean_squared_error, mean_absolute_error

%matplotlib inline

In [ ]:
# Монтирование Goodle-диска
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df_save = pd.read_excel('/content/drive/MyDrive/УИИ(marphida)/Стажировка Профпатолог/ICdata_I(71).xlsx')
dfb = pd.read_excel('/content/drive/MyDrive/УИИ(marphida)/Стажировка Профпатолог/ICdata_I(71)b.xlsx')
df0 = pd.read_excel('/content/drive/MyDrive/УИИ(marphida)/Стажировка Профпатолог/ICdata_I(71)0.xlsx')
dfb0 = pd.read_excel('/content/drive/MyDrive/УИИ(marphida)/Стажировка Профпатолог/ICdata_I(71)0b.xlsx')

In [ ]:
df = df_save.copy()

# СЛУЖЕБНЫЕ ФУНКЦИИ

In [ ]:
# Глобальные переменные
EMPTY_FIELDS = 'не определено'
### Параметр отступа от границ единичного интервала нормирования
DELTA_NORM = 0.01

## Функции подсчёта заполненности колонок

In [ ]:
# Аргументы - датафрейм и список наименований столбцов
# Исключения: NaN или EMPTY_FIELDS ("Не определено")
#
def count_columns_fill(df, arg_list):
  count_list = 0
  print('Текущая структура датафрейма: ', df.shape, '\n')
  for col_name in arg_list:
    count_list += 1
    print(f'*** {col_name}: ***')
#    print('Количество непустых элементов: ', len(df[df[col_name] != EMPTY_FIELDS]) )
#    print('Количество уникальных элементов: ', len(df.loc[df[col_name] != EMPTY_FIELDS, col_name].unique()), '\n')
    if len(df[col_name].isna()) == 0:
      print('Количество непустых элементов: ', len(df[col_name] != EMPTY_FIELDS))
#      print('Количество уникальных элементов: ', len(df.loc[df[col_name] != EMPTY_FIELDS, col_name].unique()), '\n')
      print('Количество уникальных элементов: ', df[col_name].nunique(), '\n')
    else:
      print('Количество непустых элементов: ', len(df[~df[col_name].isna()]))
#      print('Количество уникальных элементов: ', len(df.loc[~df[col_name].isnull(), col_name].unique()), '\n')
      print('Количество уникальных элементов: ', df[col_name].nunique(), '\n')
  print('Количество столбцов: ', count_list)

## Функция, возвращающая количество значащих записей и уникальных элементов в столбце

In [ ]:
'''
# Аргументы - датафрейм и строка наименования столбца
# return: count_fills, count_unique
#
def count_fill_and_unique(df, col_name):
#  return len(df[df[col_name] != EMPTY_FIELDS]), len(df.loc[df[col_name] != EMPTY_FIELDS, col_name].unique())
  return len(df[df[col_name] != EMPTY_FIELDS]), df[col_name].nunique()
'''

## Функция просмотра содержания ненулевых уникальных значений полей в выбранных столбцах

In [ ]:
# arg - наименования столбцов
def prn_unique_in_col(df, *arg, sort_asc = False):
  for colname in arg:
#    unique_in_column_list = df.loc[~df[colname].isnull(), colname].unique().astype(str)
    unique_in_column_list = df[colname].unique().astype(str)
#    count_unique_in_column_list = len(unique_in_column_list)
    count_unique_in_column_list = df[colname].nunique()
    print(f'*** "{colname}" ({count_unique_in_column_list} уникальных из {len(df[df[colname] != EMPTY_FIELDS])} непустых): ***')
    to_join = sorted(unique_in_column_list) if sort_asc else unique_in_column_list
    print('; '.join(to_join))
    print()

## Функция получения списка уникальных записей столбца исходных данных и их количества

In [ ]:
'''
# arg - наименования столбцов
def get_unique_in_col(df, colname): ##, sort_asc = False):
#  unique_in_column_list = df.loc[~df[colname].isnull(), colname].unique().astype(str)
#  count_unique_in_column_list = len(unique_in_column_list)
  unique_in_column_list = df[colname].unique().astype(str)
  count_unique_in_column_list = df[colname].nunique()
#  print('\n'.join(to_join))
#  print()
  return count_unique_in_column_list, unique_in_column_list
'''

# ПОДГОТОВКА ДАТАФРЕЙМОВ С ДАННЫМИ ДЛЯ СОЗДАНИЯ ОБУЧАЮЩИХ ВЫБОРОК
- df - обработанный датафрейм исходных данных обучения
- dfcd - датафрейм рассчитываемых категориальных и нормировочных параметров для формирования обучающих датасетов

#### Заменим все NaN на 'Не определено'

In [ ]:
df.shape

(1922, 71)

In [ ]:
df.isna().sum().sum()

85613

In [ ]:
df = df.fillna('Не определено')

In [ ]:
df.isna().sum().sum()

0

In [ ]:
df_save = df_save.fillna('Не определено')
dfb = dfb.fillna('Не определено')
df0 = df0.fillna('Не определено')
dfb0 = dfb0.fillna('Не определено')

## Очистка строковых записей датафрейма от служебных символов и концевых пробелов
- + замена "временно не годен" на "не годен"

In [ ]:
def clear_df(df):
  for i in range(df.shape[0]): #iterate over rows
    if df.dtypes[1] != 'O':
      continue
    for j in range(df.shape[1]): #iterate over columns
      value = df.iloc[i, j] #get cell value
      if isinstance(value, str):
        value_n = value.replace('\n', ' ').replace('\xa0', '').strip().lower()
        if value_n == 'временно не годен':
          df.iloc[i, j] = 'не годен'
        elif value != value_n:
          df.iloc[i, j] = value_n
  return df

## Структура датафрейма для сборки данных формирования датасетов обучения

In [ ]:
df_col_data = {
    'name':[],  # Наименование колонки
    'ind_col':[], # Индекс колонки в датафрейме исходных данных
    'fill_cnt':[], # Количество полей в колонке, имеющих определённое значение, кроме 'не определено'
    'unique_cnt':[], # Количество уникальных полей в колонке, исключая 'не определено'
    'data_type':[], # Тип данных колонки
    'df_tr':[] # Имя датафрейма с нормированными и one hot encoding значениями уникальных элементов для обучения
}
dfcd = pd.DataFrame(df_col_data)

## Создание датафрейма для сборки данных формирования датасетов обучения
- последняя колонка - датасеты с категориальными и нормировочными данными для каждого параметра исходных данных (столбца df) обучающей выборки

In [ ]:
for col in df:
  d_col = pd.DataFrame(columns=np.asarray(['col_name', 'd_norm', 'd_ohe'], dtype='object'))
  dfcd.loc[len(dfcd.index)] = [
    col,
    df.columns.get_loc(col),
    len(df[df[col] != EMPTY_FIELDS]),
    # len(df.loc[df[col] != EMPTY_FIELDS, col].unique()),
    df[col].nunique(),
    df[col].dtype,
#    'dftr' + str(df.columns.get_loc(col))
    d_col
  ]
dfcd

/usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/cast.py:948: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  element = np.asarray(element)


,name,ind_col,fill_cnt,unique_cnt,data_type,df_tr
0,КлиентДатаРождения,0,1922,1794,datetime64[ns],"Empty DataFrame Columns: [col_name, d_norm, d_..."
1,КлиентПол,1,1922,2,object,"Empty DataFrame Columns: [col_name, d_norm, d_..."
2,Контрагент,2,1922,177,object,"Empty DataFrame Columns: [col_name, d_norm, d_..."
3,Профессия,3,1922,598,object,"Empty DataFrame Columns: [col_name, d_norm, d_..."
4,ЗаключениеМК,4,1922,4,object,"Empty DataFrame Columns: [col_name, d_norm, d_..."
...,...,...,...,...,...,...
66,Офтальмология2_ПрофнепригодностьВременнаяПредс...,66,1922,3,object,"Empty DataFrame Columns: [col_name, d_norm, d_..."
67,Офтальмология2_ПрофнепригодностьВременнаяПредс...,67,1922,3,object,"Empty DataFrame Columns: [col_name, d_norm, d_..."
68,Офтальмология2_Диагноз_ЗначениеПредставление,68,1922,18,object,"Empty DataFrame Columns: [col_name, d_norm, d_..."
69,Терапия2_ПрофнепригодностьВременнаяПредставление2,69,1922,4,object,"Empty DataFrame Columns: [col_name, d_norm, d_..."


## Формирование нормированных и категориальных данных из обработанного датафрейма исходных данных для обучения

In [ ]:
df.shape

(1922, 71)

In [ ]:
dfcd.shape

(71, 6)

In [ ]:
###
df = clear_df(df)
# Обход всех столбцов датафрейма c исходными данными и
# формирование датасетов обучения
for col in df.columns:
  # count_unique_recs, unique_recs_list = get_unique_in_col(df, col)
  #
  if col == 'КлиентДатаРождения':
    continue
  # Введём переменные доступа к датафрейму параметров обучения для обрабатываемого столбца исходного датафрейма
  #
  # Индекс строки в dfcd с именем столбца из df
  dfcd_index = dfcd[dfcd['name']==col].index[0]
  #
  # Датафрейм параметров обучения для столбца из df - находится в ячейке столбца df_tr датафрейма dfcd
  # Столбцы аккумулируют информацию для обучения:
  # col_name - копии всех уникальных значений по текущему столбцу исходных данных обучения (из df)
  # d_norm - нормированный параметр для каждого уникального значения (в X_TRAIN)
  # d_ohe - ohe-массив для каждого уникального значения (в X_TRAIN)
  dfcd_tr = dfcd['df_tr'][dfcd_index]
  #
  # Сбрасываем датафрейм параметров обучения на случай повтороного вхождения в данную процедуру
  dfcd_tr.drop(dfcd_tr.index, inplace=True)
  #
  # Изменяем тип данных столбца нормировочных данных
  dfcd_tr[['d_norm']].astype(float)
  #
  # Получение списка и количества уникальных записей в столбце
#  unique_count, unique_list = get_unique_in_col(df, col)
  unique_list = df[col].unique().astype(str) # Получение списка уникальных записей в столбце
  unique_count = df[col].nunique() # Получение количества уникальных записей в столбце
  # Если среди уникальных нет "не определено", добавим
  if EMPTY_FIELDS not in unique_list:
    unique_list = np.append(unique_list, EMPTY_FIELDS)
    unique_count += 1
  #
  print(f'*** {col} (уникальных записей: {unique_count}); всего записей: {len(df[df[col] != EMPTY_FIELDS])} ***')

  #
  # Шаг нормирования для текущего параметра
  step_norm = (1 - 2 * DELTA_NORM)/(unique_count - 1)
  #
  # Стартовое значение параметра нормирования на каждой итерации обхода уникальных элементов
  next = DELTA_NORM
  #
  # Переменная для обработки ситуации когда "не определено" идёт не сначала
  next_unique = ''
  # Цикл формирования параметров обучения (нормирование и ohe) всех уникальных
  # значений текущего столбца исходных данных
  for i in range(unique_count):
    # dfcd_tr ['col_name', 'd_norm', 'd_ohe'] - заполняемый датафрейм параметров обучения
    col_name_cur = unique_list[i] # 'col_name'
    if i == 0: # Вставляем везде "не определено" на первое место
      if col_name_cur != EMPTY_FIELDS:
        next_unique = col_name_cur
        col_name_cur = EMPTY_FIELDS
    else:
      if next_unique:
        if col_name_cur == EMPTY_FIELDS:
          col_name_cur = next_unique
          next_unique = ''
          # continue
        else:
          save_unique = col_name_cur
          col_name_cur = next_unique
          next_unique = save_unique
    d_norm_cur = next
    next += step_norm
    d_ohe_cur = np.zeros((unique_count))
    d_ohe_cur[i] = 1
    dfcd_tr.loc[len(dfcd_tr.index )] = [col_name_cur, d_norm_cur, d_ohe_cur]
#

*** КлиентПол (уникальных записей: 3); всего записей: 1922 ***
*** Контрагент (уникальных записей: 178); всего записей: 1922 ***
*** Профессия (уникальных записей: 597); всего записей: 1922 ***
*** ЗаключениеМК (уникальных записей: 4); всего записей: 1515 ***
*** ВредныеФакторы (уникальных записей: 513); всего записей: 1922 ***
*** ПсихиатрияНаркология1_МКБ101 (уникальных записей: 10); всего записей: 1468 ***
*** ПсихиатрияНаркология1_ПрофнепригодностьВременнаяПредставление1 (уникальных записей: 4); всего записей: 1466 ***
*** ПсихиатрияНаркология1_ГоденНегоден_ЗначениеПредставление (уникальных записей: 3); всего записей: 1478 ***
*** ПсихиатрияНаркология1_Диагноз_ЗначениеПредставление (уникальных записей: 14); всего записей: 1429 ***
*** Офтальмология1_МКБ101 (уникальных записей: 20); всего записей: 1169 ***
*** Офтальмология1_ПрофнепригодностьВременнаяПредставление1 (уникальных записей: 4); всего записей: 1158 ***
*** Офтальмология1_Диагноз_ЗначениеПредставление (уникальных записей: 

###  Сохранение параметров обучающих выборок

In [ ]:
dfcd.to_excel('/content/drive/MyDrive/УИИ(marphida)/Стажировка Профпатолог/ICdata_I(71)dfcd.xlsx', index=False)

## Проверка нескольких таблиц с подготовленными данными для датасетов

In [ ]:
print(dfcd.iloc[68]['name'])
dfcd['df_tr'][68]

Офтальмология2_Диагноз_ЗначениеПредставление


,col_name,d_norm,d_ohe
0,не определено,0.010000,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,[h52.0] гиперметропия_x000d_,0.067647,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,[h52.2] астигматизм_x000d_ описание: астигмати...,0.125294,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,[z00.0] общий медицинский осмотр_x000d_,0.182941,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,[h52.1] миопия_x000d_,0.240588,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
5,[h50.9] косоглазие неуточненное_x000d_,0.298235,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
6,"[t94.0] последствия травм, захватывающих неско...",0.355882,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
7,_x000d_ _x000d_ 1. [h52.0] гиперметропия_x000d...,0.413529,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ..."
8,[h52.2] астигматизм_x000d_,0.471176,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."
9,[h52.2] астигматизм_x000d_ описание: сложный м...,0.528824,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [ ]:
print(dfcd.iloc[69]['name'])
dfcd['df_tr'][69]

Терапия2_ПрофнепригодностьВременнаяПредставление2


,col_name,d_norm,d_ohe
0,не определено,0.01,"[1.0, 0.0, 0.0]"
1,не годен,0.50,"[0.0, 1.0, 0.0]"
2,годен,0.99,"[0.0, 0.0, 1.0]"


# ФОРМИРОВАНИЕ ОБУЧАЮЩИХ ВЫБОРОК

In [ ]:
print(dfcd.iloc[4]['name'])
dfcd['df_tr'][4]
#df

ЗаключениеМК


,col_name,d_norm,d_ohe
0,не определено,0.010000,"[1.0, 0.0, 0.0, 0.0]"
1,годен,0.336667,"[0.0, 1.0, 0.0, 0.0]"
2,не годен,0.663333,"[0.0, 0.0, 1.0, 0.0]"
3,ограниченно годен,0.990000,"[0.0, 0.0, 0.0, 1.0]"


## Функция создания обучающих выборок

In [ ]:
# Создание выборок из глобального датафрейма *df* и датафрейма параметров обучения *dfcd*
def make_tr_arrays(df, zmk_no = True): # zmk_no "ЗаключениеМК": True  - с учётом "Не определено"
  #                                                             False - без "Не определено"
  # Заготовки массивов обучения
  x_train_norm = []
  y_train_norm = []
  x_train_ohe = []
  y_train_ohe = []
  ###
  for index, row in df.iterrows(): # Обход всех строк и столбцов исходного датафрейма
#    print('index=', index)
    # Индекс столбца
    df_col_index = 0
    # Списки накопления текущей строки выборок обучения
    x_norm = []
    x_ohe = []
    y_norm = []
    y_ohe = []
    for df_col_data in row:
      if df_col_index == 0: # тип "Дата" пока не участвует в обучении
        df_col_index += 1
        continue
      #
      # Выделяем датафрейм с готовыми обучающими выборками для текущего слолбца (переменной обучения из df) из dfcd['df_tr']
      #
      # col_name - все уникальные значения по текущему столбцу исходных данных обучения (из df)
      # d_norm - нормированный параметр для каждого уникального значения
      # d_ohe - ohe-массив для каждого уникального значения
      dfcd_tr = dfcd['df_tr'][dfcd[dfcd['ind_col']==df_col_index].index[0]]

  #    print('Формирование выборки для "', row.index[df_col_index], '"')

      # Обращение за "d_norm" по текущему значению параметра обучения
      try:
        par_norm_ = dfcd_tr[dfcd_tr.col_name==df_col_data].iloc[0, 1] # достаём значение "d_norm"
        par_ohe_ = dfcd_tr[dfcd_tr.col_name==df_col_data].iloc[0, 2] # достаём значение "d_ohe"
      except IndexError:
        print('↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓')
        print('Формирование выборки для "', row.index[df_col_index], '"')
        print(f'Значение "{df_col_data}" отсутствует в обученной модели')
        print(f'Вносим параметры значения "{EMPTY_FIELDS}". (Сохранить текущие данные по "{row.index[df_col_index]}" для дообучения...)')
        print('↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑↑')
        par_norm_ = dfcd_tr[dfcd_tr.col_name==EMPTY_FIELDS].iloc[0, 1] # достаём значение "d_norm"
        par_ohe_ = dfcd_tr[dfcd_tr.col_name==EMPTY_FIELDS].iloc[0, 2] # достаём значение "d_ohe"

      if df_col_index == 4: # тип "ЗаключениеМК" целевой, формируется отдельно
        if zmk_no == False: # Для датасета, очищенного от "Не определено" - оставить только "Годен", "Не годен", "Ограниченно годен"
          if par_norm_ < 0.5:
            par_norm_ = DELTA_NORM
          elif 0.9 > par_norm_ > 0.5:
            par_norm_ = 0.5
          par_ohe_ = np.delete(par_ohe_, 0)
        y_norm.append(par_norm_)
        y_ohe.append(par_ohe_)

      else:
        x_norm.append(par_norm_)
        x_ohe.append(par_ohe_)
      df_col_index += 1
    ###
    x_item_norm = np.hstack(x_norm)
    x_item_ohe = np.hstack(x_ohe)
    y_item_norm = np.hstack(y_norm)
    y_item_ohe = np.hstack(y_ohe)

    x_train_norm.append(x_item_norm)
    x_train_ohe.append(x_item_ohe)
    y_train_norm.append(y_item_norm)
    y_train_ohe.append(y_item_ohe)
  ###
  return np.array(x_train_norm), np.array(x_train_ohe), np.array(y_train_norm), np.array(y_train_ohe)

### Создание выборок по исходникам

In [ ]:
###

df_save = df_save.fillna('Не определено')
dfb = dfb.fillna('Не определено')
df0 = df0.fillna('Не определено')
dfb0 = dfb0.fillna('Не определено')
###
print('df_save.shape = ', df_save.shape)
print('dfb.shape = ', dfb.shape)
print('df0.shape = ', df0.shape)
print('dfb0.shape = ', dfb0.shape)
###
df_save = clear_df(df_save)
dfb = clear_df(dfb)
df0 = clear_df(df0)
fgb0 = clear_df(dfb0)
###

x_train_norm_1, x_train_ohe_1, y_train_norm_1, y_train_ohe_1 = make_tr_arrays(df_save)
print('x_train_norm_1.shape = ', x_train_norm_1.shape)
print('x_train_ohe_1.shape = ', x_train_ohe_1.shape)
print('y_train_norm_1.shape = ', y_train_norm_1.shape)
print('y_train_ohe_1.shape = ', y_train_ohe_1.shape)

x_train_norm_1b, x_train_ohe_1b, y_train_norm_1b, y_train_ohe_1b = make_tr_arrays(dfb)
print('x_train_norm_1b.shape = ', x_train_norm_1b.shape)
print('x_train_ohe_1b.shape = ', x_train_ohe_1b.shape)
print('y_train_norm_1b.shape = ', y_train_norm_1b.shape)
print('y_train_ohe_1b.shape = ', y_train_ohe_1b.shape)

x_train_norm_0, x_train_ohe_0, y_train_norm_0, y_train_ohe_0 = make_tr_arrays(df0, zmk_no = False)
print('x_train_norm_0.shape = ', x_train_norm_0.shape)
print('x_train_ohe_0.shape = ', x_train_ohe_0.shape)
print('y_train_norm_0.shape = ', y_train_norm_0.shape)
print('y_train_ohe_0.shape = ', y_train_ohe_0.shape)

x_train_norm_0b, x_train_ohe_0b, y_train_norm_0b, y_train_ohe_0b = make_tr_arrays(dfb0, zmk_no = False)
print('x_train_norm_0b.shape = ', x_train_norm_0b.shape)
print('x_train_ohe_0b.shape = ', x_train_ohe_0b.shape)
print('y_train_norm_0b.shape = ', y_train_norm_0b.shape)
print('y_train_ohe_0b.shape = ', y_train_ohe_0b.shape)

#x_train_norm = np.array(x_train_norm)
#x_train_ohe = np.array(x_train_ohe)
#y_train_norm = np.array(y_train_norm)
#y_train_ohe = np.array(y_train_ohe)

#print(x_train_norm.shape)
#print(x_train_ohe.shape)
#print(y_train_norm.shape)
#print(y_train_ohe.shape)

np.save('/content/drive/MyDrive/УИИ(marphida)/Стажировка Профпатолог/x_train_norm_1.npy', x_train_norm_1)
np.save('/content/drive/MyDrive/УИИ(marphida)/Стажировка Профпатолог/x_train_ohe_1.npy', x_train_ohe_1)
np.save('/content/drive/MyDrive/УИИ(marphida)/Стажировка Профпатолог/y_train_norm_1.npy', y_train_norm_1)
np.save('/content/drive/MyDrive/УИИ(marphida)/Стажировка Профпатолог/y_train_ohe_1.npy', y_train_ohe_1)

np.save('/content/drive/MyDrive/УИИ(marphida)/Стажировка Профпатолог/x_train_norm_1b.npy', x_train_norm_1b)
np.save('/content/drive/MyDrive/УИИ(marphida)/Стажировка Профпатолог/x_train_ohe_1b.npy', x_train_ohe_1b)
np.save('/content/drive/MyDrive/УИИ(marphida)/Стажировка Профпатолог/y_train_norm_1b.npy', y_train_norm_1b)
np.save('/content/drive/MyDrive/УИИ(marphida)/Стажировка Профпатолог/y_train_ohe_1b.npy', y_train_ohe_1b)

np.save('/content/drive/MyDrive/УИИ(marphida)/Стажировка Профпатолог/x_train_norm_0.npy', x_train_norm_0)
np.save('/content/drive/MyDrive/УИИ(marphida)/Стажировка Профпатолог/x_train_ohe_0.npy', x_train_ohe_0)
np.save('/content/drive/MyDrive/УИИ(marphida)/Стажировка Профпатолог/y_train_norm_0.npy', y_train_norm_0)
np.save('/content/drive/MyDrive/УИИ(marphida)/Стажировка Профпатолог/y_train_ohe_0.npy', y_train_ohe_0)

np.save('/content/drive/MyDrive/УИИ(marphida)/Стажировка Профпатолог/x_train_norm_0b.npy', x_train_norm_0b)
np.save('/content/drive/MyDrive/УИИ(marphida)/Стажировка Профпатолог/x_train_ohe_0b.npy', x_train_ohe_0b)
np.save('/content/drive/MyDrive/УИИ(marphida)/Стажировка Профпатолог/y_train_norm_0b.npy', y_train_norm_0b)
np.save('/content/drive/MyDrive/УИИ(marphida)/Стажировка Профпатолог/y_train_ohe_0b.npy', y_train_ohe_0b)



x_train_norm_0.shape =  (1503, 69)
x_train_ohe_0.shape =  (1503, 1987)
y_train_norm_0.shape =  (1503, 1)
y_train_ohe_0.shape =  (1503, 3)
x_train_norm_0b.shape =  (4509, 69)
x_train_ohe_0b.shape =  (4509, 1987)
y_train_norm_0b.shape =  (4509, 1)
y_train_ohe_0b.shape =  (4509, 3)


In [ ]:
len(y_train_ohe_0b[0])
y_train_ohe_0b[3]

array([0., 1., 0., 0.])

In [ ]:
a = np.array(['a', 'b', 'c'])
a = np.delete(a, 0)
a

array(['b', 'c'], dtype='<U1')

In [ ]:
# Пример обработанных данных
n = 0
print(x_train_norm[n])
print(y_train_norm[n])

[0.5        0.01553672 0.01162791 0.01191406 0.11888889 0.01
 0.5        0.08538462 0.06157895 0.01       0.01771654 0.09166667
 0.01       0.5        0.09166667 0.17333333 0.5        0.5
 0.07533333 0.09909091 0.01       0.08538462 0.5        0.01
 0.01       0.01       0.01       0.01       0.01       0.01
 0.01       0.01       0.01       0.01       0.01       0.01
 0.01       0.01       0.01       0.01       0.01       0.01
 0.01       0.01       0.01       0.01       0.01       0.01
 0.01       0.01       0.01       0.01       0.01       0.01
 0.01       0.01       0.01       0.01       0.01       0.01
 0.01       0.01       0.01       0.01       0.01       0.01
 0.01       0.01       0.01      ]
[0.33666667]


In [ ]:
# Пример обработанных данных
n = 0
print(x_train_ohe[n], f'| {len(x_train_ohe[n])} раз')
print(y_train_ohe[n])

[0. 1. 0. ... 0. 0. 0.] | 1993 раз
[0. 1. 0. 0.]


## Сохранение обучающих выборок

In [ ]:
np.save('/content/drive/MyDrive/УИИ(marphida)/Стажировка Профпатолог/x_train_norm.npy', x_train_norm)
np.save('/content/drive/MyDrive/УИИ(marphida)/Стажировка Профпатолог/x_train_ohe.npy', x_train_ohe)
np.save('/content/drive/MyDrive/УИИ(marphida)/Стажировка Профпатолог/y_train_norm.npy', y_train_norm)
np.save('/content/drive/MyDrive/УИИ(marphida)/Стажировка Профпатолог/y_train_ohe.npy', y_train_ohe)

In [ ]:
x_train_norm = np.load('/content/drive/MyDrive/УИИ(marphida)/Стажировка Профпатолог/x_train_norm.npy')
x_train_ohe = np.load('/content/drive/MyDrive/УИИ(marphida)/Стажировка Профпатолог/x_train_ohe.npy')
y_train_norm = np.load('/content/drive/MyDrive/УИИ(marphida)/Стажировка Профпатолог/y_train_norm.npy')
y_train_ohe = np.load('/content/drive/MyDrive/УИИ(marphida)/Стажировка Профпатолог/y_train_ohe.npy')

In [ ]:
df['Профессия'].where(df['Профессия'] >= 'ведущий специалист по организации обучения')

0          ведущий специалист по организации обучения
1                                                 NaN
2                                    главный менеджер
3                                  главный специалист
4                                  старший специалист
                            ...                      
1917                        директор по строительству
1918    начальник производственно-технического отдела
1919                              помощник кладовщика
1920                                            повар
1921                                              NaN
Name: Профессия, Length: 1922, dtype: object

In [ ]:
np.min(x_val)
y_train = utils.to_categorical(y_train, 10)
y_test = utils.to_categorical(y_test, 10)
y_val = utils.to_categorical(y_val, 10)


In [ ]:
for index, row in df.iterrows():
  print(index)
  print(row[4])
  break

0
годен


In [ ]:
j = 0
for index, row in df.iterrows(): # Обход всех строк и столбцов исходного датафрейма
  i = 0
  j += 1
  print(index)
  if j >= 4:
    break
  for col in row:
    print(df[])
    print(col)
    i += 1
    if i >= 5:
      print('-----')
      break

SyntaxError: ignored

In [ ]:
df[4]

In [ ]:
dfcd_tr = dfcd['df_tr'][dfcd[dfcd['name']=='ЗаключениеМК'].index[0]]
dfcd_tr

,col_name,d_norm,d_ohe
0,годен,0.010000,"[1.0, 0.0, 0.0, 0.0]"
1,не годен,0.336667,"[0.0, 1.0, 0.0, 0.0]"
2,ограниченно годен,0.663333,"[0.0, 0.0, 1.0, 0.0]"
3,не определено,0.990000,"[0.0, 0.0, 0.0, 1.0]"


In [ ]:
dfcd

,name,ind_col,fill_cnt,unique_cnt,data_type,df_tr
0,КлиентДатаРождения,0,1922,1794,datetime64[ns],"Empty DataFrame Columns: [col_name, d_norm, d_..."
1,КлиентПол,1,1922,2,object,col_name d_norm d_ohe 0 женский 0...
2,Контрагент,2,1922,177,object,col_name d...
3,Профессия,3,1922,602,object,col_...
4,ЗаключениеМК,4,1515,3,object,col_name d_norm ...
...,...,...,...,...,...,...
66,Офтальмология2_ПрофнепригодностьВременнаяПредс...,66,48,2,object,col_name d_norm d_ohe 0 ...
67,Офтальмология2_ПрофнепригодностьВременнаяПредс...,67,45,2,object,col_name d_norm d_ohe 0 ...
68,Офтальмология2_Диагноз_ЗначениеПредставление,68,51,17,object,c...
69,Терапия2_ПрофнепригодностьВременнаяПредставление2,69,73,3,object,col_name d_norm ...
